In [147]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [148]:
def to_hours(time):
    return (time // 3600) + 6

def clean_sales_data():
    df = pd.read_csv("../assets/example_data/sales_data.csv")
    sales_df = df[['token_id', 'sold_at', 'price']]
    sales_df = sales_df.rename(columns={'token_id': 'horse_id', 'sold_at': 'time_sold'}).sort_values('horse_id').reset_index(drop=True)
    sales_df['time_sold'] = sales_df['time_sold'].apply(to_hours)

    return sales_df

In [149]:
sales_data = clean_sales_data()

In [150]:
eth_prices = pd.read_csv('../assets/eth_price_history.csv')
eth_prices.tail()

,Unnamed: 0,timestamp,eth_price
9091,9091,464923,1533.01
9092,9092,464924,1529.65
9093,9093,464925,1535.38
9094,9094,464926,1538.76
9095,9095,464927,1550.97


In [151]:
merged = pd.merge(sales_data, eth_prices, left_on='time_sold', right_on='timestamp', how='left')
merged['converted_price'] = merged['price'] * merged['eth_price']
clean_market_data = merged.drop(columns=['Unnamed: 0', 'price', 'timestamp','eth_price'])

In [152]:
# horse data cleaning
horse_data = pd.read_csv('../assets/all_horse_meta.csv')
horse_data.columns
horse_data = horse_data.rename(columns={'data.horse.bloodline':'bloodline', 'data.horse.breed_type':'breed_type',
                           'data.horse.breeding_counter':'breeding_counter', 'data.horse.color':'horse_color',
                           'data.horse.horse_type':'horse_type', 'data.horse.img_url':'img_url',
                           'data.horse.last_breeding_reset':'last_breeding_reset', 'data.horse.name':'horse_name',
                           'data.horse.nft_id':'nft_id', 'data_horse_offsprings':'offsprings',
                           'data.horse.race_statistic.first_place_finishes':'first_place_finishes',
                           'data.horse.race_statistic.number_of_races':'number_races',
       'data.horse.race_statistic.second_place_finishes':'second_place_finishes',
       'data.horse.race_statistic.third_place_finishes':'third_place_finishes',
       'data.horse.race_statistic.win_rate':'win_rate'})
horse_data = horse_data.drop(columns=['Unnamed: 0'])
print(horse_data.nft_id)
horse_data.head()

0      154936
1      393233
2      131092
3      131093
4      393237
        ...  
103    131738
104       668
105    131749
106    131767
107    131778
Name: nft_id, Length: 108, dtype: int64


,bloodline,breed_type,breeding_counter,horse_color,data.horse.gen,horse_type,img_url,data.horse.inserted_at,last_breeding_reset,horse_name,nft_id,first_place_finishes,number_races,second_place_finishes,third_place_finishes,win_rate
0,Buterin,exclusive,3,Dark Byzantium,Z10,Stallion,https://img.zed.run/horses/5D3954.png,2021-11-26T20:32:39,2022-12-23T20:28:07Z,Scrunter S Thompson,154936,11,101,7,10,10.89
1,Nakamoto,exclusive,3,Fandango,Z5,Colt,https://img.zed.run/horses/B53389.png,2022-04-15T08:36:23,2022-12-23T08:31:54Z,Nakuza,393233,6,131,15,12,4.58
2,Finney,legendary,3,Air Superiority Blue,Z7,Stallion,https://img.zed.run/horses/72A0C1.png,2021-10-17T06:37:21,2023-01-08T06:30:43Z,Glorified Liberty,131092,5,65,7,8,7.69
3,Nakamoto,exclusive,3,Battle Horse Gray,Z6,Colt,https://img.zed.run/horses/848482.png,2021-10-17T06:37:43,2023-01-08T06:31:09Z,Glorified Ecstasy,131093,12,151,9,15,7.95
4,Buterin,elite,3,Eminence,Z16,Colt,https://img.zed.run/horses/6C3082.png,2022-04-15T08:51:24,2022-12-23T08:46:42Z,System America,393237,0,11,0,1,0.00


In [153]:
# code to merge horse data with sales data
test_merge = pd.merge(horse_data,clean_market_data, right_on='horse_id', left_on='nft_id', how='left')
test_merge

,bloodline,breed_type,breeding_counter,horse_color,data.horse.gen,horse_type,img_url,data.horse.inserted_at,last_breeding_reset,horse_name,nft_id,first_place_finishes,number_races,second_place_finishes,third_place_finishes,win_rate,horse_id,time_sold,converted_price
0,Buterin,exclusive,3,Dark Byzantium,Z10,Stallion,https://img.zed.run/horses/5D3954.png,2021-11-26T20:32:39,2022-12-23T20:28:07Z,Scrunter S Thompson,154936,11,101,7,10,10.89,154936,464926,15.38760
1,Buterin,exclusive,3,Dark Byzantium,Z10,Stallion,https://img.zed.run/horses/5D3954.png,2021-11-26T20:32:39,2022-12-23T20:28:07Z,Scrunter S Thompson,154936,11,101,7,10,10.89,154936,463789,11.69660
2,Nakamoto,exclusive,3,Fandango,Z5,Colt,https://img.zed.run/horses/B53389.png,2022-04-15T08:36:23,2022-12-23T08:31:54Z,Nakuza,393233,6,131,15,12,4.58,393233,460256,34.63614
3,Finney,legendary,3,Air Superiority Blue,Z7,Stallion,https://img.zed.run/horses/72A0C1.png,2021-10-17T06:37:21,2023-01-08T06:30:43Z,Glorified Liberty,131092,5,65,7,8,7.69,131092,460471,42.42640
4,Nakamoto,exclusive,3,Battle Horse Gray,Z6,Colt,https://img.zed.run/horses/848482.png,2021-10-17T06:37:43,2023-01-08T06:31:09Z,Glorified Ecstasy,131093,12,151,9,15,7.95,131093,461771,46.86210
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
151,Nakamoto,genesis,0,Thistle,Z2,Mare,https://img.zed.run/horses/D8BFD8.png,2019-03-22T04:55:23,2022-12-23T04:55:12Z,Guardian,668,88,589,68,58,14.94,668,462635,6415.40000
152,Nakamoto,genesis,0,Thistle,Z2,Mare,https://img.zed.run/horses/D8BFD8.png,2019-03-22T04:55:23,2022-12-23T04:55:12Z,Guardian,668,88,589,68,58,14.94,668,463797,5284.62000
153,Nakamoto,exclusive,3,Misty Rose,Z7,Stallion,https://img.zed.run/horses/FFE4E1.png,2021-10-18T22:54:26,2023-01-09T22:49:54Z,Ripsnorter,131749,17,309,33,30,5.50,131749,458982,117.96590
154,Finney,exclusive,3,Burnt Umber,Z16,Stallion,https://img.zed.run/horses/8A3324.png,2021-10-18T23:28:47,2023-01-09T23:24:00Z,Walk In The Park,131767,22,204,21,22,10.78,131767,459321,36.64100


## Exploratory Data Analysis